# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [103]:
import os
import requests, json
from dotenv import load_dotenv
import re
import zipfile
import io
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
load_dotenv()

True

In [104]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

MODEL_CODELLAMA = "phind/phind-codellama-34b"
MODEL_LLAMA3 = "meta-llama/llama-3-70b-instruct"
MODEL_WIZARD8x22b = "microsoft/wizardlm-2-8x22b"

# Setting up LLM

In [105]:
# Initliaze models
llm_llama3 = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_LLAMA3)
llm_codellama = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_CODELLAMA)
llm_wizard22 = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_WIZARD8x22b)

In [106]:
def get_zip_structure(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_structure = io.StringIO()
        zip_ref.printdir(file=zip_structure)
        structure_text = zip_structure.getvalue().strip()
    return structure_text

In [107]:
INSTRUCT_WRAP_CODE = "\nWrap your code in ```python\n{CODE}\n```"

In [108]:
input = "combine the images in this zip into a pdf file with one image as each page."

# input = "Convert the images in this zip file to a pdf by adding all images contained within to the PDF. Make sure to pay attention to the structure inside the zip file to make sure all code correctly handles the zipfile."

file = "images.zip"

# Generate detailed prompt from simple user prompt
template_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful prompting assistant.\
    You must write instructions for creating a python script for completing the below user prompt.\
    Your job is to expand on the instructions below to make sure there is a defined plan of execution."""),
    ("user", "User Prompt: {input}\nFile: {file}\nFile Contents: {contents}")
])

template_code = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Write code based on the instructions with redundancy for errors."),
    ("user", "Instructions: {input}\nFile: {file}\nFile Contents: {contents}")
])

file_contents = "\nContents of unzipped file:\n" + get_zip_structure(file)


def gen_content(llm_prompt, llm_code):
    prompt_chain = template_prompt | llm_prompt | output_parser
    
    code_chain = template_code | llm_code | output_parser
    
    prompt_output = prompt_chain.invoke({"input": input, "file": file, "contents": file_contents})
    
    prompt_output += INSTRUCT_WRAP_CODE
    
    code_output = code_chain.invoke({"input": prompt_output, "file": file, "contents": file_contents})
    
    python_code = re.search(r'```python(?s:(.*?))```', code_output).group(1)
    print(python_code)
    return python_code

gen_content(llm_llama3, llm_llama3)
gen_content(llm_codellama, llm_codellama)
code_output = gen_content(llm_wizard22, llm_wizard22)


import zipfile
from PIL import Image
import os
from fpdf import FPDF

try:
    # Load the zip file
    with zipfile.ZipFile('images.zip', 'r') as zip_ref:
        zip_ref.extractall('images')
except FileNotFoundError:
    print("Error: The file 'images.zip' was not found.")
    exit(1)

try:
    pdf = FPDF()
    pdf.set_page_size('A4')  # Set the page size to A4 (you can adjust this to any size you prefer)

    for filename in os.listdir('images'):
        if filename.endswith(('.jpg', '.jpeg')):
            image_path = os.path.join('images', filename)
            image = Image.open(image_path)
            width, height = image.size
            pdf.add_page()
            pdf.image(image_path, x=0, y=0, w=width, h=height)
except Exception as e:
    print("An error occurred:", str(e))
    exit(1)

try:
    pdf.output('output.pdf', 'F')
except Exception as e:
    print("An error occurred while saving the PDF file:", str(e))
    exit(1)

print("PDF file created successfully!")


import o

In [109]:
file = open(".ipynb_checkpoints/code.py", "w")
print(code_output)
file.write(str(code_output))
# print(prompt_output)


import os
import zipfile
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import shutil

# Step 1: Unzip the images
def unzip_images(zip_filename, extract_to):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Step 2: Load the images
def load_images(image_dir):
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    images = [os.path.join(image_dir, f) for f in image_files]
    return images

# Step 3: Create PDF pages
def images_to_pdf(image_files, output_pdf):
    c = canvas.Canvas(output_pdf, pagesize=letter)
    for i, image_file in enumerate(image_files):
        image = Image.open(image_file)
        width, height = letter  # Use standard letter size for PDF pages
        img_width, img_height = image.size
        aspect = img_height / float(img_width)
        new_height = int(width * aspect)
        c.drawImage(image_file, 0, 0, w

1750

In [110]:
import os
import zipfile
from PIL import Image
from fpdf import FPDF
import shutil

# Define the path to the zip file
zip_file_path = 'images.zip'

# Create a temporary directory to store the extracted images
temp_dir = 'temp_images'
os.makedirs(temp_dir, exist_ok=True)

# Extract the images from the zip file to the temporary directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Create a PDF file
pdf = FPDF()

# Iterate through the images in the temporary directory
for filename in os.listdir(temp_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Open the image using PIL
        img = Image.open(os.path.join(temp_dir, filename))

        # Get the image width and height
        width, height = img.size

        # Set the PDF page size to match the image size
        pdf.add_page(orientation='P' if width < height else 'L', format=(width, height))

        # Convert the image to a PDF and add it to the PDF
        pdf.image(os.path.join(temp_dir, filename), x=0, y=0, w=width, h=height)

# Save the PDF file
pdf.output('images.pdf')

# Remove the temporary directory
shutil.rmtree(temp_dir)
